## Instructions

1. Use krylov namespace to run the notebook. It is verified for the krylov configuration:
```json
{
  "application": "jupyterlab",
  "description": "",
  "workspaceConfiguration": {
    "image": "ecr.vip.ebayc3.com/ppetrov/krylov-passion:latest",
    "hadoop": {
      "batchUser": "b_selling_research",
      "hadoopCluster": "apollo-rno"
    }
  }
}
```

2. Setup git acces from krylov workspace (do only once)
    - Go to https://github.ebay.com/settings/tokens 
    - Click `Generate new token`, copy it
    - Open terminal in krylov workspace
    - Save the token in your home dir, e.g. in `~/.gittoken` file


3. Clone `pretrainer_utils` under some `<repos_path>` on krylov
```bash
cd <repos_path>
git clone https://`cat ~/.gittoken`@github.ebay.com/dbasin/pretrainer_utils.git
```

## Installs

In [2]:
import os
os.environ['HTTP_PROXY'] = 'http://httpproxy-tcop.vip.ebay.com:80'
os.environ['HTTPS_PROXY']='http://httpproxy-tcop.vip.ebay.com:80'

In [7]:
! pip3 install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 192.9 MB 68.9 MB/s            
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


## Imports

#### General

In [3]:
import pickle
import pandas as pd
from pyspark.sql import functions as F, Row
from fsspec.implementations import hdfs
from functools import partial

import xgboost as xgb
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'hdfs' from 'fsspec.implementations' (/opt/conda/lib/python3.10/site-packages/fsspec/implementations/__init__.py)

#### Pretrainer

In [4]:
import sys
repos_path = '/home/mmandelbrod/repositories/'
utils_path = f'{repos_path}/pretrainer_utils/utils'
sys.path.append(utils_path)

In [5]:
from pretrainer import load_npy_path, Fetcher
from spark_utils import load_spark
from pretrainer_utils import numpy_data_to_pdf, label_extract_processor, parse_category, calc_active_features, extract_label
from hdfs_utils import HDFS

In [6]:
from xgb_utils import pretrainer_train_test_split, create_dmatrix, calc_feature_imp, RecordEval, load_bst_model
from xgb_utils import calc_rank, sale_rank_stats, calc_pred_score, calc_sale_rank, calc_comb_score, model_vs_prods_ranks, calc_groups

ModuleNotFoundError: No module named 'xgboost'

## Spark setup

In [7]:
spark = load_spark(queue='hdlq-struct-default')

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/apache/releases/spark-3.1.1.1.0.0-bin-ebay/jars/parquet-kms-client-0.2.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/apache/releases/spark-3.1.1.1.0.0-bin-ebay/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


2024-09-02T18:42:58.106+0000: [GC (Allocation Failure) 
Desired survivor size 20447232 bytes, new threshold 7 (max 15)
[PSYoungGen: 122880K->14980K(142848K)] 122880K->14996K(469504K), 0.0175472 secs] [Times: user=0.04 sys=0.01, real=0.02 secs] 
2024-09-02T18:42:58.159+0000: [GC (Metadata GC Threshold) 
Desired survivor size 20447232 bytes, new threshold 7 (max 15)
[PSYoungGen: 21032K->6230K(265728K)] 21048K->6254K(592384K), 0.0062523 secs] [Times: user=0.01 sys=0.01, real=0.01 secs] 
2024-09-02T18:42:58.165+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 6230K->0K(265728K)] [ParOldGen: 24K->6148K(152064K)] 6254K->6148K(417792K), [Metaspace: 20235K->20235K(1067008K)], 0.0301337 secs] [Times: user=0.06 sys=0.00, real=0.03 secs] 


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-09-02T18:42:59.480+0000: [GC (Metadata GC Threshold) 
Desired survivor size 20447232 bytes, new threshold 7 (max 15)
[PSYoungGen: 157324K->12053K(265728K)] 163473K->18210K(417792K), 0.0126452 secs] [Times: user=0.01 sys=0.01, real=0.01 secs] 
2024-09-02T18:42:59.493+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 12053K->0K(265728K)] [ParOldGen: 6156K->17079K(216064K)] 18210K->17079K(481792K), [Metaspace: 33462K->33456K(1079296K)], 0.0627781 secs] [Times: user=0.14 sys=0.01, real=0.06 secs] 
2024-09-02T18:43:01.560+0000: [GC (Metadata GC Threshold) 
Desired survivor size 20447232 bytes, new threshold 7 (max 15)
[PSYoungGen: 219378K->19964K(409088K)] 236457K->38577K(625152K), 0.0211153 secs] [Times: user=0.03 sys=0.00, real=0.02 secs] 
2024-09-02T18:43:01.581+0000: [Full GC (Metadata GC Threshold) [PSYoungGen: 19964K->0K(409088K)] [ParOldGen: 18612K->29174K(282112K)] 38577K->29174K(691200K), [Metaspace: 56486K->56486K(1101824K)], 0.0854573 secs] [Times: user=0.18 sys=0.02, real

24/09/02 18:43:02 WARN HiveConf: DEPRECATED: hive.metastore.ds.retry.* no longer has any effect.  Use hive.hmshandler.retry.* instead
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.metastore.local does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.enforce.sorting does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.server2.proxyuser.hue.groups does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.server2.proxyuser.hue.hosts does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.metastore.ds.retry.interval does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.enforce.bucketing does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.metastore.ds.retry.attempts does not exist
24/09/02 18:43:02 WARN HiveConf: HiveConf of name hive.server2.enable.impersonation does not exist
24/09/02 18:43:04 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading li

2024-09-02T18:43:07.142+0000: [GC (Allocation Failure) 
Desired survivor size 31457280 bytes, new threshold 6 (max 15)
[PSYoungGen: 389120K->19936K(453632K)] 418294K->56127K(735744K), 0.0442534 secs] [Times: user=0.04 sys=0.06, real=0.04 secs] 


24/09/02 18:43:09 ERROR SparkContext: Error initializing SparkContext.
org.apache.hadoop.yarn.exceptions.YarnException: org.apache.hadoop.security.AccessControlException: User b_perso does not have permission to submit application_1724912156839_757521 to queue hdlq-struct-default
	at org.apache.hadoop.yarn.ipc.RPCUtil.getRemoteException(RPCUtil.java:38)
	at org.apache.hadoop.yarn.server.resourcemanager.RMAppManager.createAndPopulateNewRMApp(RMAppManager.java:513)
	at org.apache.hadoop.yarn.server.resourcemanager.RMAppManager.submitApplication(RMAppManager.java:390)
	at org.apache.hadoop.yarn.server.resourcemanager.ClientRMService.submitApplication(ClientRMService.java:742)
	at org.apache.hadoop.yarn.api.impl.pb.service.ApplicationClientProtocolPBServiceImpl.submitApplication(ApplicationClientProtocolPBServiceImpl.java:290)
	at org.apache.hadoop.yarn.proto.ApplicationClientProtocol$ApplicationClientProtocolService$2.callBlockingMethod(ApplicationClientProtocol.java:617)
	at org.apache.h

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.hadoop.yarn.exceptions.YarnException: org.apache.hadoop.security.AccessControlException: User b_perso does not have permission to submit application_1724912156839_757521 to queue hdlq-struct-default
	at org.apache.hadoop.yarn.ipc.RPCUtil.getRemoteException(RPCUtil.java:38)
	at org.apache.hadoop.yarn.server.resourcemanager.RMAppManager.createAndPopulateNewRMApp(RMAppManager.java:513)
	at org.apache.hadoop.yarn.server.resourcemanager.RMAppManager.submitApplication(RMAppManager.java:390)
	at org.apache.hadoop.yarn.server.resourcemanager.ClientRMService.submitApplication(ClientRMService.java:742)
	at org.apache.hadoop.yarn.api.impl.pb.service.ApplicationClientProtocolPBServiceImpl.submitApplication(ApplicationClientProtocolPBServiceImpl.java:290)
	at org.apache.hadoop.yarn.proto.ApplicationClientProtocol$ApplicationClientProtocolService$2.callBlockingMethod(ApplicationClientProtocol

In [33]:
# spark.stop()

In [2]:
!ls /apache


confs	    hbase-1.4	     optimus	      spark-3.1.1.1.0  zookeeper
hadoop	    hive	     pig	      spark2.3
hadoop-2.7  hive-hadoop-2.7  releases	      spark2.4
hadoop-3.3  hive-hadoop-3.3  spark	      spark3.1
hbase	    java	     spark-3.1.1.0.9  spark_bak


## Constants

#### Parameters description

- `base_path` - should  reference your model base path, e.g. `/apps/b_perso/vlp/simplark/pretrainer/RecommendedBrandOutletWithMLR` (note `no` viewfs prefix here)
- `base_out_path` - specify if you use `Extender` to add features. Defines output for numpy files generated by `Extender`. `Extender` will create a folder with run timestamp for each run.
- `start_date`,`end_date` - specify date range of loaded training data (inclusively)
- `num_workers` - number of spark executors used for fetching the data



#### Values

In [6]:
start_date = '20220801'
end_date = '20220819'

root_path = '/apps/b_perso/hp/simplark/pretrainer'
models = ['PersonalizedTopicsV2WithMetaOrganicPRecall','PersonalizedTopicsV2WithTopicMLR']
base_paths = [f'{root_path}/{m}' for m in models]

target_label = 'labelPurchase'

#### PIYI features

In [58]:
piyi_v5_features=[
  "BibowatchRelPosition",
  "RecallSourceBullseye",
  "RecallSourceTora",
  "TitleCosineSimilarityToShoppingcartCentroid",
  "FreqSameLeafCatIdInWatchBadge",
  "MaxViewedItemTitleJaccardBigrams",
  "NumSameRviInLastWeek",
  "AvgSameLeafRviPriceRatio",
  "ItemSalesOverImpPricePrior7DayDecayLogSmoothDomesticWebAndMobile",
  "ItemVariantSalesOverImpressions7DayDecayLogSmoothDomesticWebAndMobileV2",
  "MaxViewedItemTitleJaccard",
  "ItemTimeOnSiteV2",
  "ItemWatchesOverImp7DayDecayLogSmoothDomesticWebAndMobileV2",
  "PriceDiffMedianRecall",
  "FreqSameItemInWatchBadge",
  "RecallSourceBestMatch",
  "ItemSalesOverImpPricePrior7DayDecayLogSmoothInternationalWebAndMobileNorm",
  "PoissonNextEventProbSameItemInWatch",
  "FreqWatchPriceBellowItemPrice",
  "MaxSameLeafRvihPriceDiff",
  "MerchImpressionsDecayed",
  "PlImpressionsDecayed",
  "AvgSameLeafRviPriceDiff",
  "ItemSalesOverImpPricePrior7DayDecayLogSmoothDomesticWebAndMobileNorm",
  "BullseyeRelRVILeafCatMedianPriceDiffV2",
  "BullseyeAbsRVILeafCatMedianPriceDiffV2",
  "BullseyeRVILeafCatMedianPriceV2",
  "LeafCatRVICondition",
  "ItemConditionOrdinal",
  "ItemConditionNorm",
  "SameItemConditionInRvi"
]

## Inspecting sample of data

In [7]:
fetcher = Fetcher(base_paths[0], start_date, end_date, hdfs.HadoopFileSystem(), num_workers=128)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  """Entry point for launching an IPython kernel.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/apache/releases/hadoop-2.7.3.2.6.4.2.0.18/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/apache/releases/hadoop-2.7.3.2.6.4.2.0.18/share/hadoop/httpfs/tomcat/webapps/webhdfs/WEB-INF/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/apache/releases/hadoop-2.7.3.2.6.4.2.0.18/share/hadoop/kms/tomcat/webapps/kms/WEB-INF/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
22/09

In [8]:
fetcher.paths[:5]

['/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithMetaOrganicPRecall/2022/08/01/data/part-00000-21a60939-e7fe-497d-887e-e222a3c8e0c0.npy.gz',
 '/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithMetaOrganicPRecall/2022/08/01/data/part-00001-21a60939-e7fe-497d-887e-e222a3c8e0c0.npy.gz',
 '/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithMetaOrganicPRecall/2022/08/01/data/part-00002-21a60939-e7fe-497d-887e-e222a3c8e0c0.npy.gz',
 '/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithMetaOrganicPRecall/2022/08/01/data/part-00003-21a60939-e7fe-497d-887e-e222a3c8e0c0.npy.gz',
 '/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithMetaOrganicPRecall/2022/08/01/data/part-00004-21a60939-e7fe-497d-887e-e222a3c8e0c0.npy.gz']

In [9]:
len(fetcher.paths)

550

In [10]:
data = load_npy_path(fetcher.paths[0])

In [11]:
type(data)

numpy.ndarray

In [ ]:
data.dtype.descr

In [13]:
len(data['features'].dtype.names)

1363

In [14]:
calc_active_features??

Signature: calc_active_features(paths)
Docstring: <no docstring>
Source:   
def calc_active_features(paths):
    data_list = [load_npy_path(p) for p in paths]
    feature_cols = [d[0] for d in data_list[0]['features'].dtype.descr if len(d) == 2 and d[1] == '<f4']
    data_pdfs = [pd.DataFrame({c:data['features'][c] for c in feature_cols}) for data in data_list]
    samp_pdf = pd.concat(data_pdfs)
    
    features_cnts = samp_pdf.apply(lambda a: len(np.unique(a))) 
    active_feature_cols = features_cnts[features_cnts > 1].index.values
    return active_feature_cols
File:      /data/ebay/notebooks/dbasin/Repositories/pretrainer_utils/utils/pretrainer_utils.py
Type:      function


In [15]:
active_feature_cols = calc_active_features(fetcher.paths[:2])
len(active_feature_cols)

667

In [ ]:
numpy_data_to_pdf??

In [16]:
numpy_data_to_pdf(data, feature_cols=active_feature_cols)

meta                                                          \
             itemId                                 meid      userId siteId   
0      314073542053  b'0031e70828624a469b0500fd142ab59e'  2392490943      3   
1      333338270853  b'0031e70828624a469b0500fd142ab59e'  2392490943      3   
2      164781472281  b'0031e70828624a469b0500fd142ab59e'  2392490943      3   
3      394134436416  b'0031e70828624a469b0500fd142ab59e'  2392490943      3   
4      392881016823  b'0031e70828624a469b0500fd142ab59e'  2392490943      3   
...             ...                                  ...         ...    ...   
67417  303160964134  b'fff2ffec53b0445fa6f36472ebdabde6'           0      0   
67418  304555384526  b'fff2ffec53b0445fa6f36472ebdabde6'           0      0   
67419  233745264868  b'fff2ffec53b0445fa6f36472ebdabde6'           0      0   
67420  195155559280  b'fff2ffec53b0445fa6f36472ebdabde6'           0      0   
67421  384402295256  b'fff2ffec53b0445fa6f36472ebdabde6'           0      0   

                         labels                              \
      rank  category labelClick labelPurchase labelCombined   
0        0  b'63861'          0             0             0   
1        1  b'63861'          0             0             0   
2        2  b'63861'          1             0             0   
3        3  b'63861'          0             0             0   
4        4  b'63861'          0             0             0   
...    ...       ...        ...           ...           ...   
67417    5  b'33712'          0             0             0   
67418    6  b'33712'          0             0             0   
67419    7  b'33712'          0             0             0   
67420    8  b'33712'          0             0             0   
67421    9  b'33712'          0             0             0   

                                            features  ...  \
      NormItemViewCount7DayDecayDomesticWebAndMobile  ...   
0                                           0.009248  ...   
1                                           0.151473  ...   
2                                           0.028934  ...   
3                                           0.025835  ...   
4                                           0.043820  ...   
...                                              ...  ...   
67417                                       0.000642  ...   
67418                                       0.001063  ...   
67419                                       0.000000  ...   
67420                                       0.000000  ...   
67421                                       0.000182  ...   

                                                              \
      MaxSameLeafRvihPriceDiff FreqWatchPriceBellowItemPrice   
0                     1.657025                  4.467643e-09   
1                     6.397025                  2.680586e-09   
2                     5.837026                  2.680586e-09   
3                     2.747025                  3.574115e-09   
4                     6.157025                  2.680586e-09   
...                        ...                           ...   
67417                 4.900002                 -1.000000e+00   
67418                 6.990002                 -1.000000e+00   
67419                10.000000                 -1.000000e+00   
67420                26.000002                 -1.000000e+00   
67421                39.990002                 -1.000000e+00   

                                                        \
      MaxTransactionPriceRatio UserLowPricePrpnstyDiff   
0                     1.886038                    -1.0   
1                     2.936545                    -1.0   
2                     2.755237                    -1.0   
3                     2.055099                    -1.0   
4                     2.856000                    -1.0   
...                        ...                     ...   
67417                -1.000000                    -1.0   
67418             

In [18]:
label_extract_processor??

Signature:
label_extract_processor(
    data,
    target_label,
    feature_cols,
    meta_cols=['itemId', 'meid', 'userId', 'siteId', 'rank', 'category'],
    label_cols=['labelClick', 'labelPurchase', 'labelCombined'],
)
Docstring: <no docstring>
Source:   
def label_extract_processor(data, target_label, feature_cols, meta_cols = default_meta_cols,  label_cols=default_label_cols):
    label_cols = list(set(label_cols) | (set([target_label])))
    pdf = numpy_data_to_pdf(data, meta_cols, label_cols, feature_cols)
    res = extract_label(pdf, target_label)
    del pdf
    return res
File:      /data/ebay/notebooks/dbasin/Repositories/pretrainer_utils/utils/pretrainer_utils.py
Type:      function


In [17]:
label_extract_processor(data, target_label='labelPurchase', feature_cols=active_feature_cols)

meta                                                          \
             itemId                                 meid      userId siteId   
919    234506805927  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
920    195243638026  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
921    194965666729  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
922    324847563039  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
923    155086381712  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
...             ...                                  ...         ...    ...   
65933  324803642979  b'96e7e87b78dd49a8aa197ea727d5cd36'  2347710389      0   
65934  224839234785  b'96e7e87b78dd49a8aa197ea727d5cd36'  2347710389      0   
65935  224678965763  b'96e7e87b78dd49a8aa197ea727d5cd36'  2347710389      0   
65936  224611318421  b'96e7e87b78dd49a8aa197ea727d5cd36'  2347710389      0   
65937  324872379561  b'96e7e87b78dd49a8aa197ea727d5cd36'  2347710389      0   

                             labels                           \
      rank   category labelPurchase labelClick labelCombined   
919      0   b'43961'             1          1             1   
920      1   b'43961'             0          0             0   
921      2   b'43961'             0          0             0   
922      3   b'43961'             0          0             0   
923      4   b'43961'             0          0             0   
...    ...        ...           ...        ...           ...   
65933    4  b'170098'             0          0             0   
65934    5  b'170098'             0          0             0   
65935    6  b'170098'             0          0             0   
65936    7  b'170098'             0          0             0   
65937    8  b'170098'             0          0             0   

                                            features  ...  \
      NormItemViewCount7DayDecayDomesticWebAndMobile  ...   
919                                         0.003148  ...   
920                                         0.002032  ...   
921                                         0.004533  ...   
922                                         0.001133  ...   
923                                         0.000855  ...   
...                                              ...  ...   
65933                                       0.000287  ...   
65934                                       0.000455  ...   
65935                                       0.001963  ...   
65936                                       0.000597  ...   
65937                                       0.000850  ...   

                                                              \
      MaxSameLeafRvihPriceDiff FreqWatchPriceBellowItemPrice   
919                   0.000000                      0.000003   
920                   5.039997                      0.000003   
921                   1.040001                      0.000003   
922                  20.989998                      0.000003   
923                  36.989998                      0.000003   
...                        ...                           ...   
65933                17.139999                     -1.000000   
65934                16.500000                     -1.000000   
65935                16.400000                     -1.000000   
65936                16.500000                     -1.000000   
65937                17.110001                     -1.000000   

                                                        \
      MaxTransactionPriceRatio UserLowPricePrpnstyDiff   
919                  -1.000000                    -1.0   
920                  -1.000000                    -1.0   
921                  -1.000000                    -1.0   
922                  -1.000000                    -1.0   
923                  -1.000000                    -1.0   
...                        ...                     ...   
65933                20.681080                    -1.0   
65934

## Fetching train data

In [19]:
base_paths

['/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithMetaOrganicPRecall',
 '/apps/b_perso/hp/simplark/pretrainer/PersonalizedTopicsV2WithTopicMLR']

In [20]:
fetchers = [Fetcher(base_path, start_date, end_date, hdfs.HadoopFileSystem(), num_workers=128) for base_path in base_paths]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  """Entry point for launching an IPython kernel.


In [21]:
pdfs = [ft.fetch_pandas_df(spark, partial(label_extract_processor, target_label=target_label,feature_cols=active_feature_cols)) for ft  in fetchers]

In [22]:
pdf = pd.concat(pdfs, ignore_index=True)

In [23]:
pdf

meta                                                          \
              itemId                                 meid      userId siteId   
0       234506805927  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
1       195243638026  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
2       194965666729  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
3       324847563039  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
4       155086381712  b'4120b73a3201499fb48e4186efb83e4a'  2434096930      0   
...              ...                                  ...         ...    ...   
717358  175354598698  b'ded3590672d149bfba10c5eecf3f0683'   671499032     77   
717359  125465238852  b'ded3590672d149bfba10c5eecf3f0683'   671499032     77   
717360  175385450683  b'ded3590672d149bfba10c5eecf3f0683'   671499032     77   
717361  165635082173  b'ded3590672d149bfba10c5eecf3f0683'   671499032     77   
717362  175373401211  b'ded3590672d149bfba10c5eecf3f0683'   671499032     77   

                             labels                           \
       rank  category labelCombined labelClick labelPurchase   
0         0  b'43961'             1          1             1   
1         1  b'43961'             0          0             0   
2         2  b'43961'             0          0             0   
3         3  b'43961'             0          0             0   
4         4  b'43961'             0          0             0   
...     ...       ...           ...        ...           ...   
717358    5  b'31388'             0          0             0   
717359    6  b'31388'             0          0             0   
717360    7  b'31388'             0          0             0   
717361    8  b'31388'             0          0             0   
717362    9  b'31388'             0          0             0   

                                             features  ...  \
       NormItemViewCount7DayDecayDomesticWebAndMobile  ...   
0                                            0.003148  ...   
1                                            0.002032  ...   
2                                            0.004533  ...   
3                                            0.001133  ...   
4                                            0.000855  ...   
...                                               ...  ...   
717358                                       0.004635  ...   
717359                                       0.001070  ...   
717360                                       0.000892  ...   
717361                                       0.000353  ...   
717362                                       0.002883  ...   

                                                               \
       MaxSameLeafRvihPriceDiff FreqWatchPriceBellowItemPrice   
0                      0.000000                  3.051753e-06   
1                      5.039997                  3.051753e-06   
2                      1.040001                  3.051753e-06   
3                     20.989998                  3.051753e-06   
4                     36.989998                  3.051753e-06   
...                         ...                           ...   
717358               111.409424                  0.000000e+00   
717359              -243.120544                  3.089699e-10   
717360              -191.460632                  3.089699e-10   
717361              -181.330627                  3.089699e-10   
717362              -435.580627                  3.089699e-10   

                                                         \
       MaxTransactionPriceRatio UserLowPricePrpnstyDiff   
0                     -1.000000                    -1.0   
1                     -1.000000                    -1.0   
2                     -1.000000                    -1.0   
3                     -1.000000                    -1.0   
4                     -1.000000                    -1.0   
...                         ...                     ...   
717358           

In [24]:
pdf[('meta', 'meid')] = pdf.meta.meid.map(lambda v: v.decode())

In [26]:
pdf['features']

,NormItemViewCount7DayDecayDomesticWebAndMobile,ItemViewsOverImp7DayDecayLogSmoothDomesticWebAndMobileNorm,PlSellerSalesOverImpressions,ItemViewsOverImp7DayDecay,AtmometerPpm,ShippingIsPlus,ItemSaleCount7DayDecayDomesticWebAndMobile,ItemVariantWatchOverImpressions7DayDecayLogSmoothDomesticWebAndMobileNorm,ItemSaleCount7DayDecayAll,IsSellerMarketingOfferMessage,...,MaxSameLeafRvihPriceDiff,FreqWatchPriceBellowItemPrice,MaxTransactionPriceRatio,UserLowPricePrpnstyDiff,TitleCosineSimilarityCentroidRvisInLastDay,AvgWatchPriceBidRatioBadge,AvgTransactionPriceRatio,NumSameRviLeafCatInLastTwoDay,TimeSinceAddedWatch,MaxWatchPriceBinRatioBadge
0,0.003148,0.541955,-3.755054,0.008665,0.000000,0.0,2.35,0.643256,1.01,0.0,...,0.000000,3.051753e-06,-1.000000,-1.0,0.000000,2.6396,-1.000000,0.0,0.007478,-1.000000
1,0.002032,0.510849,-3.516945,0.003504,0.000000,0.0,1.48,0.683541,0.00,0.0,...,5.039997,3.051753e-06,-1.000000,-1.0,0.000000,2.4380,-1.000000,0.0,0.007478,-1.000000
2,0.004533,0.500786,-3.516945,0.007544,0.000499,0.0,3.97,0.669890,1.49,0.0,...,1.040001,3.051753e-06,-1.000000,-1.0,0.000000,2.5980,-1.000000,0.0,0.007478,-1.000000
3,0.001133,0.585978,-3.746583,0.021535,0.000000,0.0,0.77,0.559003,0.00,0.0,...,20.989998,3.051753e-06,-1.000000,-1.0,0.000000,1.8000,-1.000000,0.0,0.007478,-1.000000
4,0.000855,0.421043,-4.174220,0.002273,0.000000,0.0,0.00,0.578653,0.00,0.0,...,36.989998,3.051753e-06,-1.000000,-1.0,0.000000,1.1600,-1.000000,0.0,0.007478,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717358,0.004635,0.505994,-3.473964,0.006382,0.000000,0.0,0.00,0.559738,0.00,0.0,...,111.409424,0.000000e+00,0.026271,-1.0,0.509175,-1.0000,69.554443,2.0,37.460117,0.020227
717359,0.001070,0.351522,-3.473964,0.005162,0.000000,0.0,0.00,0.593666,0.00,0.0,...,-243.120544,3.089699e-10,0.018915,-1.0,0.406181,-1.0000,96.603050,2.0,37.460117,0.014563
717360,0.000892,0.306725,-3.473964,0.004711,0.000000,0.0,0.00,0.373950,0.00,0.0,...,-191.460632,3.089699e-10,0.019720,-1.0,0.513200,-1.0000,92.661697,2.0,37.460117,0.015183
717361,0.000353,0.448197,-3.473964,0.015793,0.000000,0.0,0.00,0.469752,0.00,0.0,...,-181.330627,3.089699e-10,0.019885,-1.0,0.449050,-1.0000,91.888832,2.0,37.460117,0.015311


## Prepare train/test data

#### All non-leaking feature cols

In [47]:
active_cols = pdf.features.columns
exclusion_cols = ['PurchasedQuantity', 
                  'InvertedRank', 
                  'InvertedRankV2',
                  'FinalScore', 
                  'LegacyMlrScoreLogStd', 
                  'LegacyMlrScoreLogMax', 
                  'MlrModelScore', 
                  'LegacyMlrScoreLogMeanInteractMax', 
                  'LegacyMlrScoreLogMean',
                  'LegacyMlrScoreLogMeanInteractMedian',
                  'LegacyMlrScoreLogMedian',
                  'LegacyMlrScoreLogMin'
                 ]
candidate_cols = list(set(active_cols) - set(exclusion_cols))

#### Input for training

In [6]:
pretrainer_train_test_split??

Signature: pretrainer_train_test_split(pdf, test_size, random_stats=7)
Docstring: <no docstring>
Source:   
def pretrainer_train_test_split(pdf, test_size, random_stats=7):
    partition_keys = np.unique(pdf['meta']['meid'])
    train_keys, valid_keys = train_test_split(partition_keys, test_size=test_size, random_state=7)
    train_set, valid_set = set(train_keys), set(valid_keys)
    train = pdf[pdf['meta']['meid'].map(lambda v: v in train_set)]
    valid = pdf[pdf['meta']['meid'].map(lambda v: v in valid_set)]
    
    return train, valid
File:      /data/ebay/notebooks/dbasin/Repositories/pretrainer_utils/utils/xgb_utils.py
Type:      function


In [27]:
train, valid = pretrainer_train_test_split(pdf, test_size=0.1)

In [28]:
valid.head(2)

meta                                                            \
          itemId                              meid      userId siteId rank   
39  313901486252  75f96c6bc16e4d94aa4a7d2b759c2fce  2411690625      3    0   
40  154213799196  75f96c6bc16e4d94aa4a7d2b759c2fce  2411690625      3    1   

                    labels                           \
    category labelCombined labelClick labelPurchase   
39  b'38659'             0          0             0   
40  b'38659'             1          1             1   

                                         features  ...  \
   NormItemViewCount7DayDecayDomesticWebAndMobile  ...   
39                                       0.010951  ...   
40                                       0.002059  ...   

                                                           \
   MaxSameLeafRvihPriceDiff FreqWatchPriceBellowItemPrice   
39                 2.272552                           0.0   
40                -1.377449                           0.0   

                                                     \
   MaxTransactionPriceRatio UserLowPricePrpnstyDiff   
39                 1.981707                    -1.0   
40                 1.783265                    -1.0   

                                                                          \
   TitleCosineSimilarityCentroidRvisInLastDay AvgWatchPriceBidRatioBadge   
39                                       -1.0                       -1.0   
40                                       -1.0                       -1.0   

                                                                               \
   AvgTransactionPriceRatio NumSameRviLeafCatInLastTwoDay TimeSinceAddedWatch   
39                 1.550552                           0.0           46.884651   
40                 1.723099                           0.0           46.884651   

                               
   MaxWatchPriceBinRatioBadge  
39                 549.032043  
40                 494.053497  

[2 rows x 676 columns]

In [29]:
candidate_cols = active_feature_cols

In [33]:
y_train =train.labels.labelPurchase
y_valid = valid.labels.labelPurchase

In [ ]:
X_train = train.features[candidate_cols]
X_valid = valid.features[candidate_cols]

# IMPORTANT !!!! train and valid must by ordered by meid for group calculations
group_train = calc_groups(train)
group_valid = calc_groups(valid)

In [49]:
eval_set = [(X_train, y_train), (X_valid, y_valid)]
eval_group = [group_train, group_valid]
eval_metric = ['map', 'ndcg@10-']
eval_result={}
# cbs = [RecordEval()]

## Simple XGB fit

In [50]:
params = {
    'objective': "rank:pairwise",
    'nthread': -1
}

In [51]:
# ranker = xgb.XGBRanker(tree_method='gpu_hist', **params)
ranker = xgb.XGBRanker(tree_method='gpu_hist', **params)

In [ ]:
model = ranker.fit(X_train, 
          y_train,
          group=group_train,
          eval_set=eval_set, 
          eval_group=eval_group, 
          eval_metric=eval_metric,
          early_stopping_rounds=50, 
#                callbacks=cbs,
               verbose=True
         )

In [53]:
top_weight_features = calc_feature_imp(model, imp_type='weight'); top_weight_features.reset_index(drop=True)[:30]

,feature,score
0,BullseyeRelRVILeafCatMedianPriceDiffV2,188.0
1,MaxCartItemTitleJaccard,93.0
2,BibowatchRelPosition,93.0
3,RecallSize,92.0
4,PriceDiffMedianRecall,91.0
5,TitleCosineSimilarityToShoppingcartCentroid,86.0
6,FreqSameLeafCatIdInWatchBadge,84.0
7,ItemWatchOverImpLogSmoothAllNorm,84.0
8,SeedItemTimeLeftSec,75.0
9,ItemTimeOnSiteV2Std,67.0


In [54]:
top_gain_features = calc_feature_imp(model, imp_type='gain'); top_gain_features.reset_index(drop=True)[:30]

,feature,score
0,BibowatchRelPosition,2255.786865
1,RecallSourceBullseye,1089.329468
2,TimeDiffFromLastRvi,1044.473633
3,MaxViewedItemTitleJaccardBigrams,805.235596
4,IsItemAuctionPure,633.687561
5,AvgSameItemWatchPriceBidDiffBadge,507.469971
6,RecallSourceTora,477.740265
7,ItemVariantSalesOverImpressions7DayDecayLogSmo...,468.413025
8,TitleCosineSimilarityToShoppingcartCentroid,413.959442
9,MaxSameItemWatchPriceRatio,281.034485


In [55]:
model_vs_prods_ranks(model, valid, candidate_cols,dmatrix=False)

,clicks,prod_clicks,purchases,prod_purchases
count,7471.000000,7471.000000,7471.000000,7471.000000
mean,2.266899,2.541695,2.533128,2.998394
std,1.976360,2.124025,2.187085,2.433809
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,2.000000,1.000000,2.000000
75%,3.000000,3.000000,3.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000


## Eval saved models

In [56]:
m = xgb.Booster()
m.load_model('./simplex_piyi_v5.bst')

[12:31:23] WARNING: ../src/learner.cc:940: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


In [ ]:
model_vs_prods_ranks(m, valid, piyi_v5_features)